In [11]:
from scamp import test_run, Session, settings, wait
from config import ARTIFACTS_RELPATH
from scamp_utils import set_default_scamp_settings
from scamp_extensions.pitch import Scale, ScaleType, hertz_to_midi
from numbers import Real
from operator import itemgetter
from itertools import chain
from common_utils.supercollider import SuperCollider
import numpy as np

set_default_scamp_settings()


In [12]:
HOST = "supercollider"
session = Session().run_as_server()
session.tempo = 65

supercollider = SuperCollider("../artifacts/scd_runtime.json", HOST)
connection_params = {
    "port": supercollider.get_lang_port(),
    "ip_address": HOST,
}
inst1 = session.new_osc_part(
    "scamp1", port=supercollider.get_lang_port(), ip_address=HOST
)
inst2 = session.new_osc_part(
    "scamp1", port=supercollider.get_lang_port(), ip_address=HOST
)


In [13]:
def bach_bwv_147_10():
    # fmt: off
    # JS Bach, BWV 147 No.10 - Jesu, Joy of Man's Desiring
    bach_bwv_147_10_sequence = np.array( [
            [ 0, 2, 4, 7, 5, 5, 9, 7, 7, 12, 11, 12, 7, 4, 0, 2, 4, ],
            # [ 5, 7, 9, 7, 5, 4, 2, 4, 0, -1, 0, 2, -5, -1, 2, 5, 4, 2, 4, ],
            # [ 0, 2, 4, 7, 5, 5, 9, 7, 7, 12, 11, 12, 7, 4, ],
            # [ 0, 2, 4, -5, 7, 5, 4, 2, 0, -5, 0, -1, 0, 4, 7, 12, 7, 4, ],
        ],
        dtype=object,
    )
    # fmt: on
    sequence = [48 + i for i in np.hstack(bach_bwv_147_10_sequence).tolist()]
    velocities = np.repeat(0.8, len(sequence)).tolist()
    durations = np.repeat(1 / 3, len(sequence)).tolist()

    for pitch, velocity, duration in zip(sequence, velocities, durations):
        inst1.play_note(
            pitch,
            velocity,
            duration,
            [{"articulations": "staccato"}],
        )
        # wait(duration)


In [21]:
def some_bass_line(note_duration: float):
    sequence = [48 + i for i in [0, 2, 4]] * 6
    velocities = np.repeat(0.8, len(sequence)).tolist()
    durations = np.repeat(note_duration, len(sequence)).tolist()

    for pitch, velocity, duration in zip(sequence, velocities, durations):
        inst2.play_note(
            pitch,
            velocity,
            duration,
            [{"articulations": "staccato"}, "pitch - 12"],
        )
        # wait(duration)


In [20]:
session.fork(bach_bwv_147_10)
session.fork(some_bass_line, args=[1/2])
session.wait_for_children_to_finish()


In [ ]:
# sequence = [0, 5, 7, 13, 5, 7, 1] * 4
# sequence = [48 + i for i in sequence]
# for root in sequence:
#     chord = [root + i for i in range(0, 13, 2)]
#     inst.play_chord(chord, 0.8, 2)
#     wait(2)
